In [261]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [262]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [263]:
features_with_na = [features for features in train_df.columns if train_df[features].isnull().sum()>1]
features_with_na

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [264]:
for feature in features_with_na:
    print(feature, np.round(train_df[feature].isnull().mean(), 4), "%missing values")

LotFrontage 0.1774 %missing values
Alley 0.9377 %missing values
MasVnrType 0.5973 %missing values
MasVnrArea 0.0055 %missing values
BsmtQual 0.0253 %missing values
BsmtCond 0.0253 %missing values
BsmtExposure 0.026 %missing values
BsmtFinType1 0.0253 %missing values
BsmtFinType2 0.026 %missing values
FireplaceQu 0.4726 %missing values
GarageType 0.0555 %missing values
GarageYrBlt 0.0555 %missing values
GarageFinish 0.0555 %missing values
GarageQual 0.0555 %missing values
GarageCond 0.0555 %missing values
PoolQC 0.9952 %missing values
Fence 0.8075 %missing values
MiscFeature 0.963 %missing values


In [265]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [266]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [267]:
catagorical_features = [feature for feature in train_df.columns if train_df[feature].dtypes == 'O']

In [268]:
catagorical_features

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [269]:
for i in catagorical_features:
    print("The feature is {} and the unique values are {}".format(i, len(train_df[i].unique())))

The feature is MSZoning and the unique values are 5
The feature is Street and the unique values are 2
The feature is Alley and the unique values are 3
The feature is LotShape and the unique values are 4
The feature is LandContour and the unique values are 4
The feature is Utilities and the unique values are 2
The feature is LotConfig and the unique values are 5
The feature is LandSlope and the unique values are 3
The feature is Neighborhood and the unique values are 25
The feature is Condition1 and the unique values are 9
The feature is Condition2 and the unique values are 8
The feature is BldgType and the unique values are 5
The feature is HouseStyle and the unique values are 8
The feature is RoofStyle and the unique values are 6
The feature is RoofMatl and the unique values are 8
The feature is Exterior1st and the unique values are 15
The feature is Exterior2nd and the unique values are 16
The feature is MasVnrType and the unique values are 4
The feature is ExterQual and the unique v

In [270]:
train_df = train_df.drop(['Alley'], axis=True)
test_df = test_df.drop(['Alley'], axis=True)
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [271]:
mean_value_train = train_df['LotFrontage'].mean()
mean_value_test = test_df['LotFrontage'].mean()
train_df['LotFrontage'].fillna(value=mean_value_train, inplace=True)
test_df['LotFrontage'].fillna(value=mean_value_test, inplace=True)
train_df.head()

C:\Users\rutur\AppData\Local\Temp\ipykernel_12400\2423558834.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['LotFrontage'].fillna(value=mean_value_train, inplace=True)
C:\Users\rutur\AppData\Local\Temp\ipykernel_12400\2423558834.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [272]:
train_df = train_df.drop(['Id'], axis=True)
test_df = test_df.drop(['Id'], axis=True)

In [273]:
train_df = train_df.drop(['MiscFeature', 'PoolQC'], axis=True)
test_df = test_df.drop(['MiscFeature', 'PoolQC'], axis=True)

In [274]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [275]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['MSZoning_label'] = le.fit_transform(train_df['MSZoning'])
train_df.head()
train_df = train_df.drop(['MSZoning'],axis=True)
test_df['MSZoning_label'] = le.fit_transform(test_df['MSZoning'])
test_df.head()
test_df = test_df.drop(['MSZoning'],axis=True)

In [276]:
# train_df = train_df.drop(['color_label'], axis=True)
# train_df.head()

In [277]:
train_df['Street'].unique()

array(['Pave', 'Grvl'], dtype=object)

In [278]:
street_map = {'Pave':1, 'Grvl':2}
train_df['Street'] = train_df['Street'].map(street_map)
train_df.head()
test_df['Street'] = test_df['Street'].map(street_map)

In [279]:
train_df['LotShape'].unique()

array(['Reg', 'IR1', 'IR2', 'IR3'], dtype=object)

In [280]:
train_df['LotShape_label'] = le.fit_transform(train_df['LotShape'])
train_df = train_df.drop(['LotShape'], axis=True)
test_df['LotShape_label'] = le.fit_transform(test_df['LotShape'])
test_df = test_df.drop(['LotShape'], axis=True)

In [281]:
train_df['LandContour'].unique()

array(['Lvl', 'Bnk', 'Low', 'HLS'], dtype=object)

In [282]:
list1 = ['MSZoning','Street','Alley','LotShape','PoolQC','MiscFeature','BldgType','HouseStyle']
for col in list1:
    catagorical_features.remove(col)
catagorical_features

['LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'Fence',
 'SaleType',
 'SaleCondition']

In [283]:
for i in catagorical_features:
    for j in train_df.columns:
        if i== j:
            train_df[j+'_label'] = le.fit_transform(train_df[j])

In [284]:
for i in catagorical_features:
    for j in test_df.columns:
        if i== j:
            test_df[j+'_label'] = le.fit_transform(test_df[j])

In [285]:
train_df.head()

,MSSubClass,LotFrontage,LotArea,Street,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,Functional_label,FireplaceQu_label,GarageType_label,GarageFinish_label,GarageQual_label,GarageCond_label,PavedDrive_label,Fence_label,SaleType_label,SaleCondition_label
0,60,65.0,8450,1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,6,5,1,1,4,4,2,4,8,4
1,20,80.0,9600,1,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,6,4,1,1,4,4,2,4,8,4
2,60,68.0,11250,1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,6,4,1,1,4,4,2,4,8,4
3,70,60.0,9550,1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,6,2,5,2,4,4,2,4,8,0
4,60,84.0,14260,1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,6,4,1,1,4,4,2,4,8,4


In [286]:
for i in catagorical_features:
    for j in train_df.columns:
        if i==j:
            train_df = train_df.drop(j,axis=True)

In [287]:
for i in catagorical_features:
    for j in test_df.columns:
        if i==j:
            test_df = test_df.drop(j,axis=True)

In [288]:
train_df.head()

,MSSubClass,LotFrontage,LotArea,Street,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,Functional_label,FireplaceQu_label,GarageType_label,GarageFinish_label,GarageQual_label,GarageCond_label,PavedDrive_label,Fence_label,SaleType_label,SaleCondition_label
0,60,65.0,8450,1,1Fam,2Story,7,5,2003,2003,...,6,5,1,1,4,4,2,4,8,4
1,20,80.0,9600,1,1Fam,1Story,6,8,1976,1976,...,6,4,1,1,4,4,2,4,8,4
2,60,68.0,11250,1,1Fam,2Story,7,5,2001,2002,...,6,4,1,1,4,4,2,4,8,4
3,70,60.0,9550,1,1Fam,2Story,7,5,1915,1970,...,6,2,5,2,4,4,2,4,8,0
4,60,84.0,14260,1,1Fam,2Story,8,5,2000,2000,...,6,4,1,1,4,4,2,4,8,4


In [289]:
train_df['BldgType_label'] = le.fit_transform(train_df['BldgType'])
train_df = train_df.drop(['BldgType'], axis=True)
test_df['BldgType_label'] = le.fit_transform(test_df['BldgType'])
test_df = test_df.drop(['BldgType'], axis=True)

In [290]:
train_df['HouseStyle_label'] = le.fit_transform(train_df['HouseStyle'])
train_df = train_df.drop(['HouseStyle'], axis=True)
test_df['HouseStyle_label'] = le.fit_transform(test_df['HouseStyle'])
test_df = test_df.drop(['HouseStyle'], axis=True)

In [291]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        layers.Dense(64,activation='relu', input_shape = [76], kernel_initializer='he_normal'),
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ]
)

In [292]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='mse', optimizer=optimizer)
data = train_df
train_df = train_df.drop(['SalePrice'],axis=True)
X_train = train_df
y_train = data['SalePrice']
X_test = test_df

In [293]:
test_df.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'Street', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'MSZoning_label', 'LotShape_label',
       'LandContour_label', 'Utilities_label', 'LotConfig_label',
       'LandSlope_label', 'Neighborhood_label', 'Condition1_label',
       'Condition2_label', 'RoofStyle_label', 'RoofMatl_label',
       'Exterior1st_label', 'Exterior2nd_label', 'MasVnrType_label',
       'ExterQual_label', 'ExterCond_label', 'Foundation_label',
       'BsmtQual_label', 'BsmtCond_label', 'BsmtExposure_label',
 

In [294]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit and transfrom
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# everything has been scaled between 1 and 0
print('Max: ',X_train.max())
print('Min: ', X_train.min())

Max:  nan
Min:  nan


In [295]:
model.fit(X_train,y_train,
         batch_size = 512,
         epochs = 100,
         )

Epoch 1/100
3/3 [==============================] - 1s 14ms/step - loss: nan
Epoch 2/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: nan
Epoch 5/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 7/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 8/100
3/3 [==============================] - 0s 13ms/step - loss: nan
Epoch 9/100
3/3 [==============================] - 0s 14ms/step - loss: nan
Epoch 10/100
3/3 [==============================] - 0s 13ms/step - loss: nan
Epoch 11/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 12/100
3/3 [==============================] - 0s 11ms/step - loss: nan
Epoch 13/100
3/3 [==============================] - 0s 10ms/step - loss: nan
Epoch 14

In [296]:
predictions = model.predict(X_test)

46/46 [==============================] - 0s 4ms/step


In [297]:
predictions

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)